In [1]:
import sys
sys.path.insert(0,r"/usr/local/lib/python3.7/site-packages")
sys.path.insert(0,r"/Users/guoliang/Library/Python/3.7/lib/python/site-packages")

import numpy as np
import matplotlib.pyplot as plt
import copy
import torch

# import scripts
from scripts.api_helper import transformer
from scripts.api_helper import fake_api
from scripts.api_helper import api_utils
from scripts.api_helper import portfolio_constructor
from scripts.exp_helper import exp_helper
from scripts.Botorch_opt import bayesian_optimiser
from scripts.function_slicer import slicer
from scripts import optutil
from datetime import datetime as dt, date as dt_date
import pandas as pd

dtype = torch.float

##### portfolio construction

In [8]:
import datetime
cobdate = datetime.date(2020, 8, 11)
positions = []
for i in range(10):
    a = np.random.randint(1,10, (6,3))
    positions.append(a)
positions[-1]

array([[5, 8, 3],
       [5, 6, 4],
       [1, 1, 2],
       [8, 4, 7],
       [8, 8, 8],
       [4, 5, 7]])

In [3]:
dfs = portfolio_constructor.construct_portfolio(positions)
dfs[0]

,Barclays - F&O Clearing,Barclays - 3952C F&O,BAML - F&O Clearing
BTSU0,9,4,1
DUU0,9,4,3
IKU0,6,2,3
OATU0,4,8,3
OEU0,6,1,2
RXU0,6,4,7


In [4]:
wrap = optutil.OptimisationWrap(cobdate, dfs[0], randomize = False)

In [5]:
wrap.x0

array([0.64285714, 0.8       , 0.5625    , 0.57142857, 0.54545455,
       0.4       , 0.26666667, 0.72727273, 0.66666667, 0.33333333,
       0.35294118, 0.36363636])

In [6]:
wrap.df(wrap.x0)

,Barclays - F&O Clearing,Barclays - 3952C F&O,BAML - F&O Clearing
BTSU0,9,4,1
DUU0,9,4,3
IKU0,6,2,3
OATU0,4,8,3
OEU0,6,1,2
RXU0,6,4,7


##### data

In [ ]:
position = np.array([ 
    [10,21,39,12,14],
    [42,52,63,11,54],
    [71,82,99,42,57],
    [15,24,67,78,93],
]) 
N = np.sum(position,axis=1)
x0 = transformer.x_to_y(position) # change of variable
x0

In [ ]:
# initial margin
fake_api.fake_margin(x0.flatten())

In [ ]:
# time horizon
T = 5
ndim = 8 # input dimension

# intial samples; assume 1 data point; ndim-dimensional input
x = torch.from_numpy(x0.flatten()[:ndim]).float().view(-1,ndim) # shape [n,d]
y = torch.tensor(-1,dtype=dtype).view(-1,1) # shape [n,1]; min f = max -f
m0 = fake_api.fake_margin(x0.flatten()[:ndim])[0]

data = [(x,y,m0), (x,y,m0)]

def api(x,m0):
    x = x.numpy()
    margins = fake_api.fake_margin(x)
    record = [None] * len(margins)
    for i,margin in enumerate(margins):
        record[i] = -(margin/m0)
    return torch.tensor(record, dtype = dtype).view(-1,1)
        
print(x)
print(y)
print(m0)

##### visualisation

In [ ]:
scr = slicer(ndim)
slices = scr.slice_query(api, m0, [2,3])
len(slices)

In [ ]:
# scr.plot(slices, 1, np.arange(.8, 1.1,0.05))

##### BO

In [ ]:
# hyperparameters:

# gp
gp_name,gp_params = "MA",{"nu":2.5,
                          "power":2,
                          "opt":"quasi_newton", # opt for GP hyper-parameter tuning
                          "epochs": 100, # if chosen ADAM, it requires training epoch
                          "lr": 1e-1, # learning rate for ADAM
                         } 
# q-parallelism
batch_size = 2

hyper_params = {
    "acq_name" : "qUCB",  # acqu func
    "N_start": 16, # number of starts for multi-start SGA
    "raw_samples" :256, # heuristic initialisation 
    "N_MC_sample" : 64, # number of samples during Monte Carlo simulation
    "num_fantasies": 64, # number of fantasies used by KG
    "beta": 1e-1, # used by UCB/ qUCB
    "candidate_set":(200,8), # used by MES; (numbers of points, input_dim)
    "MES_num_fantasies": 16, # used by MES
}

bayes_opt = bayesian_optimiser(gp_name,gp_params,**hyper_params)

In [ ]:
xs, ys, init_margins, _ = bayes_opt.random_start_exp(T, data, api, batch_size)

In [ ]:
exp_helper.show_exp_results(xs, ys, init_margins, folder_name = "", save_name = "")